# 네이버 뉴스 기사크롤링

In [119]:
#independency
from bs4 import BeautifulSoup
import urllib.request

In [120]:
# 출력 파일 명
OUTPUT_FILE_NAME = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/신문기사/output.txt'

# 긁어 올 URL
URL = 'https://www.mk.co.kr/news/stock/view/2018/03/187616/'

# 크롤링 함수
def get_text(URL):
    source_code_from_URL = urllib.request.urlopen(URL)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='Content'):
        text = text + str(item.find_all(text=True))
    return text

# 메인 함수
def main():
    open_output_file = open(OUTPUT_FILE_NAME, 'w')
    result_text = get_text(URL)
    open_output_file.write(result_text)
    open_output_file.close()
    
 
if __name__ == '__main__':
    main()

In [8]:
""" 텍스트 정제 모듈
    영어, 특수기호 모두 제거
"""
 
import re
 
# 입,출력 파일명
INPUT_FILE_NAME = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/신문기사/raw/output.txt'
OUTPUT_FILE_NAME = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/신문기사/cleaning/output_cleand.txt'
 
 
# 클리닝 함수
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    return cleaned_text
    
 
# 메인 함수
def main():
    read_file = open(INPUT_FILE_NAME, 'r')
    write_file = open(OUTPUT_FILE_NAME, 'w')
    text = read_file.read()
    text = clean_text(text)
    write_file.write(text)
    read_file.close()
    write_file.close()
 
 
if __name__ == "__main__":
    main()

In [60]:
""" 동아일보 특정 키워드를 포함하는, 특정 날짜 이전 기사 내용 크롤러(정확도순 검색)
    python [모듈 이름] [키워드] [가져올 페이지 숫자] [결과 파일명]
    한 페이지에 기사 15개
"""
 
import sys
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
 
TARGET_URL_BEFORE_PAGE_NUM = "http://news.donga.com/search?p="
TARGET_URL_BEFORE_KEWORD = '&query='
TARGET_URL_REST = '&check_news=1&more=1&sorting=3&search_date=1&v1=&v2=&range=3'
 
 
# 기사 검색 페이지에서 기사 제목에 링크된 기사 본문 주소 받아오기
def get_link_from_news_title(page_num, URL, output_file):
    for i in range(page_num):
        current_page_num = 1 + i*15
        position = URL.index('=')
        URL_with_page_num = URL[: position+1] + str(current_page_num) \
                            + URL[position+1 :]
        source_code_from_URL = urllib.request.urlopen(URL_with_page_num)
        soup = BeautifulSoup(source_code_from_URL, 'lxml',
                             from_encoding='utf-8')
        for title in soup.find_all('p', 'tit'):
            title_link = title.select('a')
            article_URL = title_link[0]['href']
            get_text(article_URL, output_file)
 
 
# 기사 본문 내용 긁어오기 (위 함수 내부에서 기사 본문 주소 받아 사용되는 함수)
def get_text(URL, output_file):
    source_code_from_url = urllib.request.urlopen(URL)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    content_of_article = soup.select('div.article_txt')
    for item in content_of_article:
        string_item = str(item.find_all(text=True))
        output_file.write(string_item)
 
 
# 메인함수
def main(argv):
    if len(argv) != 4:
        print("python [모듈이름] [키워드] [가져올 페이지 숫자] [결과 파일명]")
        return
    keyword = argv[1]
    page_num = int(argv[2])
    output_file_name = argv[3]
    target_URL = TARGET_URL_BEFORE_PAGE_NUM + TARGET_URL_BEFORE_KEWORD \
                 + quote(keyword) + TARGET_URL_REST
    output_file = open(output_file_name, 'w')
    get_link_from_news_title(page_num, target_URL, output_file)
    output_file.close()
 
 
if __name__ == '__main__':
    main(sys.argv)

python [모듈이름] [키워드] [가져올 페이지 숫자] [결과 파일명]


# 조선일보 크롤링

In [10]:
#independency
import sys
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote

In [21]:
#URL을 가져온 후 text를 가져오는 함수 정의
def get_text(URL, output_file):
    
    #urllib로 news 기사 page요청받기
    source_code_from_url = urllib.request.urlopen(URL)
    
    #BeautiffulSoup을 통해 url로 부터 source code 가져오기
    soup = BeautifulSoup(source_code_from_url, 'html.parser')
    soup.encoding = 'utf-8'
    
    #기사의 본문 내용을 추출
    content_of_article = soup.select('#news_body_id > div') #추출한 URL에 기사의 본문이 있는 위치
    
    for item in content_of_article:
        string_item = str(item.find_all(text = True))
        output_file.write(string_item)

In [17]:
#URL을 가져오는 함수
def crawlling(page, category):
    page = int(page)
    
    for page in range(1, page+1):
        #제목이 있는 URL
        URL_with_page_num = 'http://news.chosun.com/svc/list_in/list.html?catid='+str(category)+'&pn='+str(page)
        source_code_from_URL = urllib.request.urlopen(URL_with_page_num)
        soup = BeautifulSoup(source_code_from_URL, 'html.parser')
        soup.encoding = 'utf-8'
        
        i = 1
    
        for title in soup.find_all('div', attrs={'class': 'petitionsView_title'}):
            title_link = title.select('a')
            output_file_name = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/신문기사/raw/{category}_page{page}_{i}.txt'.format(category = category, page=page, i = i)
            output_file = open(output_file_name, 'w', -1, 'utf-8')
            article_URL = title_link[0]['href']
            get_text(article_URL, output_file)
            i += 1
            
        output_file.close()

In [22]:
crawlling(1,1)

# 네이버 기사 크롤링

In [23]:
#independency
import sys
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote

In [142]:
#URL을 가져오는 함수

name = 'LG%20CNS'
start_date = '2018.01.01'
end_date = '2018.03.31'
page = '2'

url1 = 'https://search.naver.com/search.naver?where=news&query='+name
url2 = '&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds='+start_date+'&de='+end_date
url3 = '&docid=&nso=so:r,p:from'+start_date+'to'+end_date+',a:all&mynews=0&cluster_rank=38&start='+page


def crawlling(name, start_date, end_date, page):
    page = int(page)
    
    for page in range(1, page+1):
        
        #제목이 있는 URL
        URL_with_page_num = url1 + url2 + url3
        source_code_from_URL = urllib.request.urlopen(URL_with_page_num)
        soup = BeautifulSoup(source_code_from_URL, 'html.parser')
        soup.encoding = 'utf-8'
        
        i = 1
        
        for title in soup.find_all('dt'):
            title_link = title.select('a')
            output_file_name = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/신문기사/raw/{name}_{start_date}_{end_date}_page{page}_{i}.txt'.format(name = name, start_date = start_date, end_date = end_date, page=page, i = i)
            output_file = open(output_file_name, 'w', -1, 'utf-8')
            article_URL = title_link[0]['href']
            get_text(article_URL, output_file)
            i += 1
            
        output_file.close()

In [143]:
#URL을 가져온 후 text를 가져오는 함수 정의
def get_text(URL, output_file):
    
    #urllib로 news 기사 page요청받기
    source_code_from_url = urllib.request.urlopen(URL)
    
    #BeautiffulSoup을 통해 url로 부터 source code 가져오기
    soup = BeautifulSoup(source_code_from_url, 'html.parser')
    soup.encoding = 'utf-8'
    
    #기사의 본문 내용을 추출
    content_of_article = soup.select('tr >td > div > p') #추출한 URL에 기사의 본문이 있는 위치
    
    for item in content_of_article:
        string_item = str(item.find_all(text = True))
        output_file.write(string_item)

# 매일경제 기사 크롤링

In [1]:
#independency
import sys
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import re

In [16]:
#URL 가져오는 함수
name = ['%BB%EF%BC%BASDI','LG%B5%F0%BD%BA%C7%C3%B7%B9%C0','%B1%E2%BE%C6%C2%F7', 'LG%C0%FC%C0%DA', '%BB%EF%BC%BA%BB%FD%B8%ED', '%C4%DA%BF%FE%C0%CC',
        '%C7%F6%B4%EB%C1%A6%C3%B6','%C7%F6%B4%EB%C1%DF%B0%F8%BE%F7']
year = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2008]
#'%BB%EF%BC%BASDI' #삼성SDI
#'LG%B5%F0%BD%BA%C7%C3%B7%B9%C0' #LG디스플레이 LG%B5%F0%BD%BA%C7%C3%B7%B9%C0%CC
#'%B1%E2%BE%C6%C2%F7' #기아차
#'LG%C0%FC%C0%DA' #LG전자
#'%BB%EF%BC%BA%BB%FD%B8%ED' #삼성생명
#'%C4%DA%BF%FE%C0%CC' #웅진코웨이
#'%C7%F6%B4%EB%C1%A6%C3%B6' #현대제철
#'%C7%F6%B4%EB%C1%DF%B0%F8%BE%F7' #현대중공업
    
def crawlling(name, year):
    page = 35
    start_date = '&y1='+str(year)+'&m1=01&d1=01'
    end_date = '&y2='+str(year)+'&m2=12&d2=31'

    
    for page in range(1, page+1):
        page = str(page)
        
        url1 = 'http://find.mk.co.kr/new/search.php?pageNum='+page
        url2 = '&cat=&cat1=&media_eco=&pageSize=10&sub=all&dispFlag=OFF&page=news&s_kwd='+name+'&s_page=news&go_page=&ord=1&ord1=1&ord2=0&s_keyword='+name+'&period=p_direct&s_i_keyword='+name
        url3 = '&s_author='+start_date+end_date+'&media%5B1%5D=00001&ord=1&area=tt'

        #제목이 있는 URL
        URL_with_page_num = url1 + url2 + url3
        source_code_from_url = urllib.request.urlopen(URL_with_page_num)
        soup = BeautifulSoup(source_code_from_url, 'html.parser')
        soup.encoding = 'utf-8'
        
        i = 1
        
        for element in soup.find_all('div', 'sub_list'):
            date = element.find_all('span', 'art_time')
            title_link = element.select('a')
            article_URL = title_link[0]['href']
            dates = re.findall("\d+", date[0].text)
            dates = dates[:3]
            dates = '_'.join(dates)
            output_file_name = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/data/신문기사/매일경제/'+str(name)+'/'+str(year)+'/{date}_page{page}_{i}.txt'.format(date  =dates, page=page, i = i)
            output_file = open(output_file_name, 'w', -1, 'utf-8')
            get_text(article_URL, output_file)
            i += 1
            
        output_file.close()
        
        

In [17]:
#URL을 가져온 후 text를 가져오는 함수 정의
def get_text(URL, output_file):
    
    #urllib로 news 기사 page요청받기
    source_code_from_url = urllib.request.urlopen(URL)
    
    #BeautiffulSoup을 통해 url로 부터 source code 가져오기
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    
    #기사의 본문 내용을 추출
    string_item = ''
    for item in soup.select('div.art_txt'):
        string_item += str(item.find_all(text=True))
        string_item = re.sub('[a-zA-Z]', '', string_item)
        string_item = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', string_item)
        output_file.write(string_item)

In [35]:
crawlling('%B1%E2%BE%C6%C2%F7',2017)

In [10]:
    crawlling('LG%B5%F0%BD%BA%C7%C3%B7%B9%C0%CC',2008)

# 한국경제 기사 크롤링 

In [1]:
#independency
import sys
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import re
from urllib.error import HTTPError

In [2]:
#URL 가져오는 함수
name = ['lg%EC%A0%84%EC%9E%90',
        '%EC%82%BC%EC%84%B1%EC%83%9D%EB%AA%85', '%EC%BD%94%EC%9B%A8%EC%9D%B4',
        '%ED%98%84%EB%8C%80%EC%A0%9C%EC%B2%A0','%ED%98%84%EB%8C%80%EC%A4%91%EA%B3%B5%EC%97%85','%EA%B8%B0%EC%95%84%EC%B0%A8']
year = [2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008]

#'%BB%EF%BC%BASDI' #삼성SDI
#'LG%EB%94%94%EC%8A%A4%ED%94%8C%EB%A0%88%EC%9D%B4' #LG디스플레이 
#'%EA%B8%B0%EC%95%84%EC%B0%A8' #기아차
#'lg%EC%A0%84%EC%9E%90' #LG전자
#'%EC%82%BC%EC%84%B1%EC%83%9D%EB%AA%85' #삼성생명
#'%EC%BD%94%EC%9B%A8%EC%9D%B4' #웅진코웨이
#'%ED%98%84%EB%8C%80%EC%A0%9C%EC%B2%A0' #현대제철
#'%ED%98%84%EB%8C%80%EC%A4%91%EA%B3%B5%EC%97%85' #현대중공업



def crawlling(name, year):
    page = 35
    start_date = str(year)+'.01.01'
    end_date = str(year)+'.12.31'
    
    for page in range(1, page+1):
        page = str(page)
        
        url1 = 'https://search.hankyung.com/apps.frm/search.news?query='+name
        url2 = '&sort=DATE%2FDESC%2CRANK%2FDESC&period=DATE&area=title&mediaid_clust=HKPAPER&sdate=' + start_date + '&edate=' + end_date
        url3 = '&exact=&include=&except=&page=' + page
        
        #제목이 있는 URL
        URL_with_page_num = url1 + url2 + url3
        source_code_from_url = urllib.request.urlopen(URL_with_page_num)
        soup = BeautifulSoup(source_code_from_url, 'html.parser')
        soup.encoding = 'utf-8'
        
        i = 1
        
        for element in soup.find_all('div', 'txt_wrap'):
            date = element.find_all('span', 'date_time')
            title_link = element.select('a')
            article_URL = title_link[0]['href']
            dates = re.findall("\d+", date[0].text)
            dates = dates[:3]
            dates = '_'.join(dates)
            output_file_name = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/data/신문기사/한국경제/'+str(name)+'/'+str(year)+'/{date}_page{page}_{i}.txt'.format(date  =dates, page=page, i = i)
            output_file = open(output_file_name, 'w', -1, 'utf-8')
            article_URL = title_link[0]['href']
            get_text(article_URL, output_file)
            i += 1
            
        output_file.close()
            

In [3]:
#URL을 가져온 후 text를 가져오는 함수 정의
def get_text(URL, output_file):
    global source_code_from_url
    
    #urllib로 news 기사 page요청받기
    try:
        source_code_from_url = urllib.request.urlopen(URL)
    except HTTPError as e:
        pass

    #BeautiffulSoup을 통해 url로 부터 source code 가져오기
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    
    #기사의 본문 내용을 추출
    string_item = ''
    for item in soup.find_all('div', id = 'articletxt'):
        string_item += str(item.find_all(text=True))
        string_item = re.sub('[a-zA-Z]', '', string_item)
        string_item = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', string_item)
        output_file.write(string_item)
        output_file.close()

In [13]:
crawlling('lg%EC%A0%84%EC%9E%90', 2008)